In [3]:
import chromadb
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pyprojroot import here
from dotenv import load_dotenv
load_dotenv()

True

### Document DB connection

In [14]:
collection_name = "imdb_sample"
chroma_client = chromadb.PersistentClient(path=str(here("data/db/chroma")))
collection = chroma_client.get_collection(collection_name)

InvalidCollectionException: Collection imdb_sample does not exist.

### Embedding client

In [4]:
embedding_client = OpenAIEmbeddings(
    model="text-embedding-3-small",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

### LLM Client

In [6]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


### Question -> Similarity search -> Response

In [11]:


system_role = """Given the following user question, and corresponding results from the vector databse, answer the user question.\n
If the vector search results are not related to the question, say "I don't know".\n

Question: {question}\n
Vector search results: {result}\n
Answer:
"""
 
answer_prompt = PromptTemplate.from_template(system_role)
answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(embedding=lambda x: embedding_client.embed_query(x["question"]))
    .assign(vector_search=lambda x: collection.query(query_embeddings=x["embedding"], n_results=1))
    .assign(result=lambda x: str(x["vector_search"]["documents"]))
    .assign(answer=answer)
)

In [12]:
question = "Can you tell me something about tv shows?"

response = chain.invoke({"question": question})
print(response["result"])
print(response["answer"])

[['Something about tv shows']]
I don't know
